# STARTUP SUCCESS PREDICTION

## Problem Statement

**Startup** adalah sesuatu bisnis yang baru saja berdiri dan berkembang dengan didukung oleh layanan digital dan juga telah menjadi elemen penting dari sistem inovasi dan ekonomi di seluruh dunia. Ekosistem **Startup** berkembang sangat pesat dan masih butuh banyak pendanaan untuk beroperasi dengan kelompok kerja yang minimalis. Jadi sangat penting bagi pihak VC untuk memantau kinerja dan performa dari **Startup**, sehingga dapat digunakan sebagai bahan pertimbangan untuk memutuskan apakah akan mendanai suatu Startup untuk mendorong pertumbuhannya atau menolak mengambil andil dalam pendanaan. Untuk memantau kinerja **Startup**, penting untuk menganalisis apa yang membuat sebuah Startup sukses dan bagaimana menentukan kesuksesannya.

## Goals
Tujuan yang ingin dicapai yaitu dapat menentukan apakah suatu StartUp akan sukses atau tidak.

## Objective
Objective yang ingin dilakukan yaitu dengan membuat analisa behaviour StartUp berdasarkan beberapa variabel yang ada, menentukan variabel apa yang paling mempengaruhi kesuksesan StartUp, kemudian membangun model yang dapat memprediksi kesuksesan dari suatu StartUp.

# Data Exploration

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import date
from scipy import stats
from scipy.stats import norm, skew #for some statistics

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
plt.style.use('seaborn')

from collections import Counter
import datetime
import wordcloud
import json

## Load Dataset

In [ ]:
df = pd.read_csv('../input/startup-success-prediction/startup data.csv')

In [ ]:
df.head(10)

## Description

In [ ]:
df.info()

## Data type identification

In [ ]:
df.columns

## Data numeric

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=df.select_dtypes(include=numeric)
df_num.head(3)

## Data categorical

In [ ]:
df_cat=df.select_dtypes(include='object')
df_cat.head(3)

### Changing 'status' data value

In [ ]:
df['status'] = df.status.map({'acquired':1, 'closed':0})
df

### Handling 'status' data type to int 

In [ ]:
#Tipe data status diganti dari object ke int
df['status'].astype(int)

### Drop column labels

In [ ]:
#labels dan status
for index, row in df.iterrows():
    if row['labels']!=row['status']:
        print(index, row['labels'], row['status'])

In [ ]:
 df.drop(["labels"], axis=1, inplace=True)

# Exploratory Data Analysis

## Pendekatan Numerik

### Statistical Summary

In [ ]:
describeNum = df.describe(include =['float64', 'int64', 'float', 'int'])
describeNum.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

In [ ]:
describeNumCat = df.describe(include=["O"])
describeNumCat.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)


- Apakah nilai yang tertera pada setiap kolom masuk akal?
- age_first_funding_year dan age_last_funding_year memiliki data min,max yang sama, perlu dicek lagi
- Apakah nilai maksimal/minimal masih berada di batas wajar?
- Min/max yang terlalu jauh dari mean/median bisa jadi indikasi kesalahan input data
- Apakah ada kolom dengan perbedaan yang signifikan antara mean dan median?
- Perbedaan antara mean/median mengindikasikan outlier atau skewed distribution

### Categorical Value Counting

In [ ]:
cats = ['state_code','zip_code','id','city','Unnamed: 6','name','founded_at','closed_at','first_funding_at','last_funding_at','state_code.1','category_code','object_id','status'] 
for col in cats:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

## Missing Value 

In [ ]:
null=pd.DataFrame(df.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1) 

- **Hasil Analisis dalam dataset yang digunakan terdapat Missing Values diantaranya yaitu**
    - **Total Missing Values yaitu 1386**
    - **Kolom yang memiliki lebih dari 50% dari missing values**
        - Variabel 'closed_at' dengan total persentase sebesar 63.70% atau total 588 kolom.
        - Variabel 'Unnamed: 6' dengan total persentase sebesar 53.41% atau total 493 kolom.
    - **Kolom yang memiliki kurang dari 50% dari missing values**
        - Variabel 'age_first_milestone_year' dengan total persentase sebesar 16.46% atau total 152 kolom.
        - Variabel 'age_last_milestone_year' dengan total persentase sebesar 16.46% atau total 152 kolom.

In [ ]:
# Checking Missing Values Column 
df[["Unnamed: 6", "closed_at", "age_first_milestone_year", "age_last_milestone_year", "state_code.1", "status"]].head(4)

- **Hasil Analisis pada kolom yang terdapat Missing Values diantaranya yaitu**
    - **Kolom "Unnamed: 6"** merupakan kolom informasi dari gabungan beberapa tabel diantaranya yaitu
        - Kolom "city", "state_code", dan "zip_code" 
    - **Kolom "closed_at"** merupakan kolom dimana StartUp **"Closed"** sehingga data yang kosong memang harusnya merupakan StarUp yang statusnya masih **"Acquired"**
    - **Kolom age_first_milestone_year** merupakan informasi kapan milestone pertama kali di lakukan dalam satuan tahun
        - Kolom ini total memiliki 771 baris data dengan Mean 3.055353 dan median 2.520500 yang memperlihatkan distribusi datanya tidak normal
    - **Kolom age_Last_milestone_year** merupakan informasi kapan milestone terakhir kali di lakukan dalam satuan tahun
        - Kolom ini total memiliki 771 baris data dengan Mean 4.754423 dan median 4.476700 yang memperlihatkan distribusi datanya tidak normal

###  Handling Missing Value Unnamed: 6

berdasarkan hasil analisis diperoleh bahwa kolom **Unnamed: 6** merupakan kombinasi dari beberapa kolom lainnya diantaranya kolom **city, state_code, dan zip_code**, maka kami memutuskan bahwa menghapus isi dari kolom **Unnamed: 6** terlebih dahulu dan kemudian mengisi data berdasarkan kombinasi dari beberapa kolom terkait.

In [ ]:
df['Unnamed: 6'] = df.apply(lambda row: (row.city) + " " + (row.state_code) + " " +(row.zip_code)  , axis = 1)
df.head()

In [ ]:
# Total Missing Values kolom "Unnamed: 6"
totalNull = df['Unnamed: 6'].isnull().sum()

print('Total Missing Values Kolom "Unnamed: 6": ', totalNull)

### Handling Missing Value closed_at

In [ ]:
#diisi kosong
df['closed_at'] = df['closed_at'].fillna(value="31/12/2013")

In [ ]:
#datetoday
#option
# df['closed_at']=df['closed_at'].fillna(date.today())


In [ ]:
totalNull = df['closed_at'].isnull().sum()

print('Total Missing Values Kolom "closed_at": ', totalNull)

### Handling Missing Value age_first_milestone_year and age_last_milestone_year

In [ ]:
df[['age_first_milestone_year','age_last_milestone_year','milestones']].head()

berdasarkan hasil analisis diperoleh bahwa kolom 'age_first_milestone_year' dan 'age_last_milestone_year' memiliki nilai null karena memang startup tersebut belum memiliki milestones. hal ini bisa dikonfirmasi dengan melihat kolom 'milestones' yang berisi data 0 pasti diiringi dengan kolom 'age_first_milestone_year' dan 'age_last_milestone_year' yang Null. sehingga kami memutuskan untuk mengisi kolom null tersebut dengan nilai 0.

In [ ]:
df['age_first_milestone_year'] = df['age_first_milestone_year'].fillna(value="0")
df['age_last_milestone_year'] = df['age_last_milestone_year'].fillna(value="0")

### Handling Missing Value state_code.1

- kolom **"state_code"** and kolom **"state_code.1"** mestilah sama, jadi kolom **"state_code.1"** harus di drop.
- kolom **"state_code.1"** memiliki missing value dibaris 515. 

In [ ]:
for index, row in df.iterrows():
    if row['state_code']!=row['state_code.1']:
        print(index, row['state_code'], row['state_code.1'])

In [ ]:
df.drop(["state_code.1"], axis=1, inplace=True)

In [ ]:
null=pd.DataFrame(df.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1) 

## Pendekatan Grafis

### Correlation heatmap

Sekarang bagaimana korelasi antar variabel data.

Korelasi direpresentasikan sebagai nilai antara -1 dan +1 di mana +1 menunjukkan korelasi positif tertinggi, -1 menunjukkan korelasi negatif tertinggi, dan 0 menunjukkan tidak ada korelasi.

In [ ]:
df.corr()

In [ ]:
df['age_first_milestone_year'] = df.age_first_milestone_year.astype(float)
df['age_last_milestone_year'] = df.age_last_milestone_year.astype(float)

In [ ]:
features = ['age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year','relationships','funding_rounds','funding_total_usd','milestones','is_CA','is_NY','is_MA','is_TX','is_otherstate','is_software','is_web','is_mobile','is_enterprise','is_advertising','is_gamesvideo','is_ecommerce','is_biotech','is_consulting','is_othercategory','has_VC','has_angel','has_roundA','has_roundB','has_roundC','has_roundD','avg_participants','is_top500','status']

plt.figure(figsize=(30,20))
ax = sns.heatmap(data = df[features].corr(),cmap='YlGnBu',annot=True)

bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5,top - 0.5)

In [ ]:
# 'SalePrice' correlation matrix (zoomed heatmap style)

#number of variables for heatmap
cols = df[features].corr().nlargest(10,'status')['status'].index
cm = np.corrcoef(df[cols].values.T) 
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, cmap='YlGnBu', fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### Scatter plot

berdasarkan tabel korelasi di atas yang mengatakan bahwa **'views'** dan **'likes'** berkorelasi sangat positif. maka kami kemudian memverifikasi itu dengan memplot scatter plot antara **'views'** dan **'likes'** untuk memvisualisasikan hubungan antara variabel-variabel tersebut.

In [ ]:
fig, ax = plt.subplots()
_ = plt.scatter(x=df['age_first_funding_year'], y=df['age_last_funding_year'], edgecolors="#000000", linewidths=0.5)
_ = ax.set(xlabel="age_first_funding_year", ylabel="age_last_funding_year")

Kami melihat bahwa **'age_first_funding_year'** dan **'age_last_funding_year'** benar-benar berkorelasi positif dimana saat satu meningkat, yang lain juga meningkat — sebagian besar.

In [ ]:
fig, ax = plt.subplots()
_ = plt.scatter(x=df['has_roundB'], y=df['funding_rounds'], edgecolors="#000000", linewidths=0.5)
_ = ax.set(xlabel="has_roundB", ylabel="funding_rounds")

In [ ]:
fig, ax = plt.subplots()
_ = plt.scatter(x=df['has_roundC'], y=df['funding_rounds'], edgecolors="#000000", linewidths=0.5)
_ = ax.set(xlabel="has_roundC", ylabel="funding_rounds")

In [ ]:
fig, ax = plt.subplots()
_ = plt.scatter(x=df['status'], y=df['relationships'], edgecolors="#000000", linewidths=0.5)
_ = ax.set(xlabel="status", ylabel="relationships")

In [ ]:
fig, ax = plt.subplots()
_ = plt.scatter(x=df['status'], y=df['milestones'], edgecolors="#000000", linewidths=0.5)
_ = ax.set(xlabel="status", ylabel="milestones")

### Box plots

In [ ]:
featuresNum = ['age_first_funding_year','age_last_funding_year','relationships','funding_rounds','funding_total_usd','milestones','avg_participants']

plt.figure(figsize=(15, 7))
for i in range(0, len(featuresNum)):
    plt.subplot(1, len(featuresNum), i+1)
    sns.boxplot(y=df[featuresNum[i]], color='green', orient='v')
    plt.tight_layout()

### Dataset collection founded years

In [ ]:
cdf = df["founded_at"].apply(lambda x: '' + x[:2]).value_counts() \
            .to_frame().reset_index() \
            .rename(columns={"index": "year", "founded_at": "No_of_startup"})

fig, ax = plt.subplots()
_ = sns.barplot(x="year", y="No_of_startup", data=cdf, 
                palette=sns.color_palette(['#003f5c', '#ffa600'], n_colors=7), ax=ax)
_ = ax.set(xlabel="Year", ylabel="No. of startup")

In [ ]:
df["founded_at"].apply(lambda x: '20:' + x[:2]).value_counts(normalize=False)

In [ ]:
df["founded_at"].apply(lambda x: '20:' + x[:2]).value_counts(normalize=True)

total ada 563 startup atau 60% startup yang berdiri pada tahun 2001

In [ ]:
df["closed_at"].apply(lambda x: '20:' + x[:2]).value_counts(normalize=True)

dari total data yang ada sebanyak 63% startup masih berdiri sedangkan sisanya 37% sudah closed dan terbanyak closed pada tahun 2001. 

### How many Startup are acquired or closed have?

In [ ]:
df_acquired = df[(df["status"] == True)]
df_acquired.shape

In [ ]:
df_closed = df[(df["status"] == False)]
df_closed.shape

In [ ]:
value_counts = df["status"].value_counts().to_dict()
fig, ax = plt.subplots()
_ = ax.pie(x=[value_counts[False], value_counts[True]], labels=['No', 'Yes'], 
           colors=['#003f5c', '#ffa600'], textprops={'color': '#040204'})
_ = ax.axis('equal')
_ = ax.set_title('Startup Acquired')

### invest feature acquired check

In [ ]:
coba = df[(df["status"] == 1)]

In [ ]:
features = coba[["has_VC","has_angel","has_roundA","has_roundB","has_roundC","has_roundD"]]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

fig, ax = plt.subplots(figsize=(12,8))

a= np.random.choice(["{}".format(i) for i in [1,2,3,4,5,6]], size=(12,8))
coba = pd.DataFrame(a, columns=["has_{}".format(i) for i in list("features")])

sns.countplot(x="variable", hue="value",palette="nipy_spectral", data=pd.melt(features))

plt.show()

### Which category has the largest number of startup

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="category_code", hue="status", data=df, palette="nipy_spectral",
              order=df.category_code.value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="Category", ylabel="No. of startup")
plt.legend(bbox_to_anchor=(0.945, 0.90))

### Which State having most number of Startup

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="state_code", hue="status", data=df, palette="nipy_spectral",
              order=df.state_code.value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="state_code", ylabel="No. of startup")
plt.legend(bbox_to_anchor=(0.945, 0.90))

In [ ]:
trending_statea = df.groupby(['state_code']).size().rename('num_startup').reset_index()

most_trending_statea = trending_statea[trending_statea.groupby('state_code')['num_startup'].transform(max) == trending_statea['num_startup']]
most_trending_statea = most_trending_statea.sort_values('num_startup', ascending=False)
most_trending_statea

### Which State having most number of acquired Startup per category

In [ ]:
trending_statea = df_acquired.groupby(['state_code','category_code']).size().rename('num_startup').reset_index()

most_trending_statea = trending_statea[trending_statea.groupby('state_code')['num_startup'].transform(max) == trending_statea['num_startup']]
most_trending_statea = most_trending_statea.sort_values('num_startup', ascending=False)
most_trending_statea.head(10)

### Which State having most number of closed Startup per category

In [ ]:
trending_statec = df_closed.groupby(['state_code','category_code']).size().rename('num_startup').reset_index()

most_trending_statec = trending_statec[trending_statec.groupby('state_code')['num_startup'].transform(max) == trending_statec['num_startup']]
most_trending_statec = most_trending_statec.sort_values('num_startup', ascending=False)
most_trending_statec

### Which city having most number of acquired Startup per category

In [ ]:
trending_categorya = df_acquired.groupby(['city','category_code']).size().rename('num_startup').reset_index()

most_trending_categorya = trending_categorya[trending_categorya.groupby('city')['num_startup'].transform(max) == trending_categorya['num_startup']]
most_trending_categorya = most_trending_categorya.sort_values('num_startup', ascending=False)
most_trending_categorya

### Which city having most number of closed Startup per category

In [ ]:
trending_categoryc = df_closed.groupby(['city','category_code']).size().rename('num_startup').reset_index()

most_trending_categoryc = trending_categoryc[trending_categoryc.groupby('city')['num_startup'].transform(max) == trending_categoryc['num_startup']].reset_index()
most_trending_categoryc = most_trending_categoryc.sort_values('num_startup', ascending=False)
most_trending_categoryc

### Which category having most number of total funding

In [ ]:
funding_sorted_category = pd.pivot_table(df,
              index=['category_code'],
              values=['funding_total_usd'],
              aggfunc=['sum']
              ).reset_index()
funding_sorted_category.columns = ['category_code', 'funding_total_usd']
funding_sorted_category = funding_sorted_category.sort_values(['funding_total_usd'], ascending = False)
funding_sorted_category.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
_ = sns.barplot(x="category_code", y="funding_total_usd", data=funding_sorted_category,
                palette="nipy_spectral", ax=ax)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="Category", ylabel="Total Funding USD")

### Which city having most number of total funding

In [ ]:
funding_sorted_city = pd.pivot_table(df,
              index=['city'],
              values=['funding_total_usd'],
              aggfunc=['sum']
              ).reset_index()
funding_sorted_city.columns = ['city', 'funding_total_usd']
funding_sorted_city = funding_sorted_city.sort_values(['funding_total_usd'], ascending = False)
funding_sorted_city = funding_sorted_city.head(10)
funding_sorted_city

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
_ = sns.barplot(x="city", y="funding_total_usd", data=funding_sorted_city,
                palette="nipy_spectral", ax=ax)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="No of State", ylabel="Number of Start Up")

In [ ]:
df_what_in_kirkland = df[(df["city"] == 'Kirkland')]
df_what_in_kirkland.shape

In [ ]:
df_what_in_kirkland.head()

Round A, Round B,Round C, Round D, VC, Angel = 0 tadi status startup acquired ?????????
terdapat hal aneh dari data yang ini, kemungkinan data tidak valid

### How many Startup have has_VC?

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

_ = sns.countplot(x="has_VC", hue="status", data=df, palette="nipy_spectral",
              order=df.has_VC.value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="Has_VC", ylabel="No. of startup")
plt.legend(bbox_to_anchor=(0.945, 0.90))

### How many Startup have is_top500?

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

_ = sns.countplot(x="is_top500", hue="status", data=df, palette="nipy_spectral",
              order=df.is_top500.value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="is_top500", ylabel="No. of startup")
plt.legend(bbox_to_anchor=(0.945, 0.90))

In [ ]:
#How many Startup have both 'acquired' status and is_top500?
len(df[(df["status"] == True) & (df["is_top500"] == True)].index)

In [ ]:
#How many Startup have both 'closed' status and is_top500?
len(df[(df["status"] == False) & (df["is_top500"] == False)].index)

In [ ]:
df_acquired["is_top500"].value_counts(normalize=True)

### How many years on average the company closes

In [ ]:
df_closed.founded_at=pd.to_datetime(df_closed.founded_at)
df_closed.closed_at=pd.to_datetime(df_closed.closed_at)

In [ ]:
df_closed['age_closed_startup'] = df_closed.apply(lambda row: (row.closed_at - row.founded_at) , axis=1)

In [ ]:
#df_closed['age_closed_startup'] = pd.to_numeric(df['age_closed_startup'].dt.days, downcast='int64')

In [ ]:
df_closed['age_closed_startup'].head()

In [ ]:
df_closed['year'] = df_closed['age_closed_startup'].dt.days /365

In [ ]:
df_closed.head(3)

In [ ]:
(df_closed['age_closed_startup'].mean()) 

In [ ]:
ratarata = round(2184 / 365) 
print("Rata-Rata Startup Closed :", ratarata ,"tahun")

### which relationship related to acquired or closed startup?

In [ ]:
fig, ax = plt.subplots(figsize=(17,10))

sns.countplot(x="relationships", hue="status", data=df, palette="nipy_spectral",
              order=df.relationships.value_counts().index)
plt.legend(bbox_to_anchor=(0.945, 0.90))

### which funding_rounds related to acquired or closed startup?

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.countplot(x="funding_rounds", hue="status", data=df, palette="nipy_spectral",
              order=df.funding_rounds.value_counts().index)
# plt.legend(bbox_to_anchor=(0.945, 0.90))

### Mapping area startup 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapefile as shp

In [ ]:
import sys

In [ ]:
'geopandas' in sys.modules

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [ ]:
print(gdf.head())

# Data Preprocessing

## Duplicate Values

In [ ]:
duplicate = df[df.duplicated()] 
  
print("Duplicate Rows :")

## Outliers

In [ ]:
#ganti data type ke numerical untuk boxplot
df['age_first_milestone_year'] = df.age_first_milestone_year.astype(float)
df['age_last_milestone_year'] = df.age_last_milestone_year.astype(float)

In [ ]:
featuresNumfinal = ['age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year','funding_total_usd','avg_participants']

plt.figure(figsize=(15, 7))
for i in range(0, len(featuresNumfinal)):
    plt.subplot(1, len(featuresNumfinal), i+1)
    sns.boxplot(y=df[featuresNumfinal[i]], color='green', orient='v')
    plt.tight_layout()

In [ ]:
def drop_outliers(df, field_name):
    iqr = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > (iqr + np.percentile(df[field_name], 75))].index, inplace=True)
    df.drop(df[df[field_name] < (np.percentile(df[field_name], 25) - iqr)].index, inplace=True)

In [ ]:
# drop_outliers(df, 'age_first_funding_year')
# drop_outliers(df, 'age_last_funding_year')
# drop_outliers(df, 'age_first_milestone_year')
# drop_outliers(df, 'age_last_milestone_year')
drop_outliers(df, 'funding_total_usd')

In [ ]:
# featuresNumfinal = ['age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year','funding_total_usd']

# plt.figure(figsize=(15, 7))
# for i in range(0, len(featuresNumfinal)):
#     plt.subplot(1, len(featuresNumfinal), i+1)
#     sns.boxplot(y=df[featuresNumfinal[i]], color='green', orient='v')
#     plt.tight_layout()

## Negative value

In [ ]:
age=["age_first_funding_year","age_last_funding_year","age_first_milestone_year","age_last_milestone_year"]

for a in range(len(age)):
    print("Is there any negative value in '{}' column  : {} ".format(age[a],(df[age[a]]<0).any()))

In [ ]:
df=df.drop(df[df.age_first_funding_year<0].index)
df=df.drop(df[df.age_last_funding_year<0].index)
df=df.drop(df[df.age_first_milestone_year<0].index)
df=df.drop(df[df.age_last_milestone_year<0].index)

In [ ]:
# # Get the absolute value of columns including negative value
# df["age_first_funding_year"]=np.abs(df["age_first_funding_year"])
# df["age_last_funding_year"]=np.abs(df["age_last_funding_year"])
# df["age_first_milestone_year"]=np.abs(df["age_first_milestone_year"])
# df["age_last_milestone_year"]=np.abs(df["age_last_milestone_year"])

In [ ]:
for a in range(len(age)):
    print("Is there any negative value in '{}' column  : {} ".format(age[a],(df[age[a]]<0).any()))

## Log-transformation of the funding and milestone year variable

In [ ]:
# df["age_first_funding_year"] = np.log(df["age_first_funding_year"])
# df["age_last_funding_year"] = np.log(df["age_last_funding_year"])
# df["age_first_milestone_year"] = np.log(df["age_first_milestone_year"])
# df["age_last_milestone_year"] = np.log(df["age_last_milestone_year"])
# df["funding_total_usd"] = np.log(df["funding_total_usd"])

In [ ]:
# plt.figure(figsize=(15,7),dpi=100)

# df["age_first_funding_year"]=np.log(df["age_first_funding_year"]+1)
# plt.subplot(2,4,1)
# plt.xlabel("age_first_funding_year")
# plt.boxplot(df["age_first_funding_year"])   

# plt.subplot(2,4,5)
# sns.distplot(df["age_first_funding_year"] , color="green");


# df["age_last_funding_year"]=np.log(df["age_last_funding_year"]+1)
# plt.subplot(2,4,2)
# plt.xlabel("age_last_funding_year")
# plt.boxplot(df["age_last_funding_year"])   

# plt.subplot(2,4,6)
# sns.distplot(df["age_last_funding_year"], color="green")


# df["age_first_milestone_year"]=np.log(df["age_first_milestone_year"]+1)
# plt.subplot(2,4,3)
# plt.xlabel("age_first_milestone_year")
# plt.boxplot(df["age_first_milestone_year"])   

# plt.subplot(2,4,7)
# sns.distplot(df["age_first_milestone_year"], color="green")


# df["age_last_milestone_year"]=np.log(df["age_last_milestone_year"]+1)
# plt.subplot(2,4,4)
# plt.xlabel("age_last_milestone_year")
# plt.boxplot(df["age_last_milestone_year"])   

# plt.subplot(2,4,8)
# sns.distplot(df["age_last_milestone_year"], color="green");


### Log-transformation funding_total_usd

In [ ]:
# print(f"Skewness Co-efficient: {round(df.funding_total_usd.skew(), 3)}")
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=300)

# ## Histogram 
# sns.distplot(df['funding_total_usd'] , fit=stats.norm, ax=ax1)
# ax1.set_title('Histogram')

# ## Probability / QQPLOT
# stats.probplot(df['funding_total_usd'], plot=ax2)

# plt.show()

In [ ]:
# df["funding_total_usd"] = np.log1p(df["funding_total_usd"])

# ##### GET SKEWNESS #####
# print(f"Skewness Co-efficient: {round(df.funding_total_usd.skew(), 3)}")

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=300)

# ##### HISTOGRAM #####
# from scipy import stats
# sns.distplot(df['funding_total_usd'] , fit=stats.norm, ax=ax1)
# ax1.set_title('Histogram')

# ##### PROBABILITY / QQ PLOT #####
# stats.probplot(df['funding_total_usd'], plot=ax2)

# plt.show()

## Normalization/Standardization

## Feature encoding

## Class imbalance

# Feature Engineering

## New Column "has_RoundABCD"

In [ ]:
df['has_RoundABCD'] = np.where((df['has_roundA'] == 1) | (df['has_roundB'] == 1) | (df['has_roundC'] == 1) | (df['has_roundD'] == 1), 1, 0)
df.head()

## New Column "has_Investor"

In [ ]:
df['has_Investor'] = np.where((df['has_VC'] == 1) | (df['has_angel'] == 1), 1, 0)
df.head()

In [ ]:
len(df[(df["has_RoundABCD"] == 1)].index)

In [ ]:
len(df[ (df['has_RoundABCD']  == 1) & (df['status']  == 1) ].index)

In [ ]:
len(df)

In [ ]:
923-490

## New Column "has_Seed"

In [ ]:
df['has_Seed'] = np.where((df['has_RoundABCD'] == 0) & (df['has_Investor'] == 1), 1, 0)
df.head()

In [ ]:
df['has_Seed'] == 1

In [ ]:
len(df[(df["has_Seed"] == 1)].index)

## New Column "invalid_startup"

In [ ]:
df['invalid_startup'] = np.where((df['has_RoundABCD'] == 0) & (df['has_VC'] == 0) & (df['has_angel'] == 0), 1, 0)
df.head()

In [ ]:
len(df[(df["invalid_startup"] == 1)].index)

##  New Column "age_of_startup

In [ ]:
df.founded_at=pd.to_datetime(df.founded_at)
df.closed_at=pd.to_datetime(df.closed_at)

In [ ]:
df['age_closed_startup'] = df.apply(lambda row: (row.closed_at - row.founded_at) , axis=1)

In [ ]:
df['age_closed_startup'].head()

In [ ]:
df['startUp_age_year'] = df['age_closed_startup'].dt.days /365

In [ ]:
df.head()

## Drop unused column for modelling

In [ ]:
df = df.drop(['state_code','id','Unnamed: 6','category_code','object_id'],axis=1)
df = df.drop(['zip_code','founded_at','closed_at','first_funding_at','last_funding_at','city','name'],axis=1)
df = df.drop(['Unnamed: 0'],axis=1)
df = df.drop(['latitude','longitude'],axis=1)
df = df.drop(['geometry'],axis=1)

df = df.drop(['age_closed_startup'],axis=1)

In [ ]:
# df['age_first_milestone_year'].astype(str).astype(float)
# df['age_last_milestone_year'].astype(str).astype(float)

df['age_first_milestone_year'] = df.age_first_milestone_year.astype(float)
df['age_last_milestone_year'] = df.age_last_milestone_year.astype(float)

# Modeling

In [ ]:
#Cek categorical
cat_feature = df.select_dtypes(include='object')
cat_feature.head()

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data
# Input/independent variables
X = df.drop('status', axis = 1) # her we are droping the output feature as this is the target and 'X' is input features, the changes are not 
                                              # made inplace as we have not used 'inplace = True'

y = df['status'] # Output/Dependent variable
# train_x, test_x,train_y,test_y = train_test_split(X,y)

In [ ]:
# # Scaling the data
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X =  sc.fit_transform(X)
# X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# lets print the shapes again 
print("Shape of the X Train :", X_train.shape)
print("Shape of the y Train :", y_train.shape)
print("Shape of the X test :", X_test.shape)
print("Shape of the y test :", y_test.shape)


In [ ]:
# Model Build
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,roc_curve, auc, precision_recall_curve, f1_score
import warnings
warnings.filterwarnings('ignore')

## LGBM Classifier

##### Feature importance by LGBM

In [ ]:
import lightgbm as lgb
#lightGBM model fit
gbm = lgb.LGBMRegressor()
gbm.fit(X_train,y_train)
gbm.booster_.feature_importance()


# importance of each attribute
fea_imp_ = pd.DataFrame({'cols':X.columns, 'fea_imp':gbm.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

##### Recursive Feature Elimination(RFE)

In [ ]:
from sklearn.feature_selection import RFE
# create the RFE model and select 10 attributes
rfe = RFE(gbm, 10)
rfe = rfe.fit(X_train,y_train)

# summarize the selection of the attributes
print(rfe.support_)

# summarize the ranking of the attributes
fea_rank_ = pd.DataFrame({'cols':X.columns, 'fea_rank':rfe.ranking_})
fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True)

##### Build Model

In [ ]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(learning_rate=0.02,
                    boosting_type='gbdt', max_depth=4,  objective='binary', 
                    random_state=100,  
                  n_estimators=1000 ,reg_alpha=0, reg_lambda=1, n_jobs=-1)


clf.fit(X_train,y_train)

y_pred_lgb = clf.predict(X_test)


print("Training Accuracy :", clf.score(X_train, y_train))
print("Testing Accuracy :", clf.score(X_test, y_test))

cm = confusion_matrix(y_test, y_pred_lgb)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_pred_lgb)
print(cr)

print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_pred_lgb)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
Precision_Recall_lgbm = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_lgbm)

## XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

#train
xgb = XGBClassifier()

xgb.fit(X_train,y_train)

#predict
y_predicted_xgb = xgb.predict(X_test)

print("Training Accuracy :", xgb.score(X_train, y_train))
print("Testing Accuracy :", xgb.score(X_test, y_test))

#eval
cm = confusion_matrix(y_test, y_predicted_xgb)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_predicted_xgb)
print(cr)

print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_predicted_xgb)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_predicted_xgb)
f1 = f1_score(y_test, y_predicted_xgb)
Precision_Recall_xgb = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_xgb)

## GradientBoosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#train
gbc = GradientBoostingClassifier(learning_rate=0.02,
                    max_depth=4,
                    random_state=100, n_estimators=1000)


gbc.fit(X_train,y_train)

#predict
y_predicted_gb = gbc.predict(X_test)

print("Training Accuracy :", gbc.score(X_train, y_train))
print("Testing Accuracy :", gbc.score(X_test, y_test))

#eval
cm = confusion_matrix(y_test, y_predicted_gb)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_predicted_gb)
print(cr)


print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_predicted_gb)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_predicted_gb)
f1 = f1_score(y_test, y_predicted_gb)
Precision_Recall_gbs = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_gbs)

In [ ]:
0.922681496797328

## AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
#train
ada = AdaBoostClassifier()


ada.fit(X_train,y_train)

#predict
y_predicted_ab = ada.predict(X_test)

print("Training Accuracy :", ada.score(X_train, y_train))
print("Testing Accuracy :", ada.score(X_test, y_test))

#eval
cm = confusion_matrix(y_test, y_predicted_ab)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_predicted_ab)
print(cr)

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_predicted_ab)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("roc_auc",roc_auc)

print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_predicted_ab)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_predicted_ab)
f1 = f1_score(y_test, y_predicted_ab)
Precision_Recall_abs = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_abs)

## SVC

In [ ]:
from sklearn.svm import SVC
sv = SVC()

sv.fit(X_train,y_train)


y_pred_sv = sv.predict(X_test)

print("Training Accuracy :", sv.score(X_train, y_train))
print("Testing Accuracy :", sv.score(X_test, y_test))

cm = confusion_matrix(y_test, y_pred_sv)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_pred_sv)
print(cr)

print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_pred_sv)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_sv)
f1 = f1_score(y_test, y_pred_sv)
Precision_Recall_svc = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_svc)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train,y_train)


y_pred_rf = rf.predict(X_test)

print("Training Accuracy :", rf.score(X_train, y_train))
print("Testing Accuracy :", rf.score(X_test, y_test))

cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (3, 3)
sns.heatmap(cm, annot = True, cmap = 'YlGnBu', fmt = '.8g')
plt.show()

cr = classification_report(y_test, y_pred_rf)
print(cr)


print("------------------------------------------")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,y_pred_rf)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("ROC Curves              =",roc_auc)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_rf)
f1 = f1_score(y_test, y_pred_rf)
Precision_Recall_rfs = auc(recall, precision)
print("Precision-Recall Curves =",Precision_Recall_rfs)

# Summary

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

scores = {'LGBM':  { 
                             'precision_score': precision_score(y_test, y_pred_lgb),
                             'recall_score': recall_score(y_test, y_pred_lgb)
                         },        
                 'GradientBoosting Classifier':  { 
                             'precision_score': precision_score(y_test, y_predicted_gb),
                             'recall_score': recall_score(y_test, y_predicted_gb)
                         },
                 'Adaboost Classifier':  { 
                             'precision_score': precision_score(y_test, y_predicted_ab),
                             'recall_score': recall_score(y_test, y_predicted_ab)
                         },
                 'SVC':  { 
                             'precision_score': precision_score(y_test, y_pred_sv),
                             'recall_score': recall_score(y_test, y_pred_sv)
                         },
                 'XGBoost':  { 
                             'precision_score': precision_score(y_test, y_predicted_xgb),
                             'recall_score': recall_score(y_test, y_predicted_xgb)
                         },
                 'Random Forest':  { 
                             'precision_score': precision_score(y_test, y_pred_rf),
                            'recall_score': recall_score(y_test, y_pred_rf)
                         }
            }

In [ ]:
from sklearn.metrics import precision_score


Precision_Recall = {'LGBM':  { 
                             'Precision_Recall': Precision_Recall_lgbm
                         },        
                 'GradientBoosting Classifier':  { 
                             'Precision_Recall': Precision_Recall_gbs
                         },
                 'Adaboost Classifier':  { 
                             'Precision_Recall': Precision_Recall_abs
                         },
                 'SVC':  { 
                             'Precision_Recall': Precision_Recall_svc
                         },
                 'XGBoost':  { 
                             'Precision_Recall': Precision_Recall_xgb
                         },
                 'Random Forest':  { 
                             'Precision_Recall': Precision_Recall_rfs
                         }
            }

In [ ]:
scores = pd.DataFrame(scores)


scores.plot(kind="barh",figsize=(12, 12)).legend(loc='upper center', ncol=3, title="Machine Learning Model")

In [ ]:
Precision_Recall = pd.DataFrame(Precision_Recall)


Precision_Recall.plot(kind="barh",figsize=(15, 8)).legend(loc='upper center', ncol=3, title="Machine Learning Model")